In [1]:
from PIL import Image
import pandas as pd
import numpy as np
import cv2 as cv
from glob import glob
from sklearn.model_selection import train_test_split
from mitosplit_net import preprocessing, augmentation, evaluation, training, plotting, util
from albumentations import Compose, Rotate, RandomRotate90, HorizontalFlip, Flip, ElasticTransform, GaussNoise, RandomCrop, Resize
from tqdm import tqdm
import os, sys
from os import path
import random as r
from import_augmentation_function import import_fun, aug_fun, import_aug_fun, normalization_fun
import matplotlib.pyplot as plt
import tensorflow as tf 
import imageio

## Choice of folder and number of augmentations ##

In [2]:
files_dir = r'C:\Users\roumba\Documents\Software\deep-events\training_data'
images_dir = '220915_mtstaygold_cos7_ZEISS_fl_pos'
date, dye, cell_type, microscope, bf_fl, pos_neg = images_dir.split('_')
joined_path = os.path.join(files_dir, images_dir)
img_size = 256
data_ratio=0.1
data_split_state = None 
number_of_augmentations = 5 

## Import and Split ##
                            

In [3]:
all_image_array, all_image_array_gauss = import_fun(joined_path, files_dir, images_dir)
augmentation_data, data_val, augmentation_data_gauss, data_gauss_val =  train_test_split(all_image_array, all_image_array_gauss, 
                                                                                                       test_size=data_ratio, random_state=data_split_state)
data_aug = augmentation_data
data_gauss_aug = augmentation_data_gauss
for j in range(number_of_augmentations):
    augment_data, augment_data_gauss = aug_fun(augmentation_data, augmentation_data_gauss)
    data_aug = np.concatenate((data_aug, augment_data))
    data_gauss_aug = np.concatenate((data_gauss_aug,augment_data_gauss))


C:\Users\roumba\Documents\Software\deep-events\training_data\220915_mtstaygold_cos7_ZEISS_fl_pos\image_220915_cos7_GFP_1.ome_0.tiff
6
0
1
2
3
4
C:\Users\roumba\Documents\Software\deep-events\training_data\220915_mtstaygold_cos7_ZEISS_fl_pos\image_220915_cos7_GFP_1.ome_0gauss.tiff
6
C:\Users\roumba\Documents\Software\deep-events\training_data\220915_mtstaygold_cos7_ZEISS_fl_pos\image_220915_cos7_GFP_1.ome_1.tiff
1
C:\Users\roumba\Documents\Software\deep-events\training_data\220915_mtstaygold_cos7_ZEISS_fl_pos\image_220915_cos7_GFP_1.ome_10.tiff
15
0
1
2
3
4
5
6
7
8
9
10
11
12
13
C:\Users\roumba\Documents\Software\deep-events\training_data\220915_mtstaygold_cos7_ZEISS_fl_pos\image_220915_cos7_GFP_1.ome_10gauss.tiff
15
C:\Users\roumba\Documents\Software\deep-events\training_data\220915_mtstaygold_cos7_ZEISS_fl_pos\image_220915_cos7_GFP_1.ome_11.tiff
1
C:\Users\roumba\Documents\Software\deep-events\training_data\220915_mtstaygold_cos7_ZEISS_fl_pos\image_220915_cos7_GFP_1.ome_11gauss.tiff
1

100%|██████████| 166/166 [00:00<00:00, 1431.47it/s]


## Normalization and Shuffle ##

In [35]:
data_val_norm, data_aug_norm, data_gauss_val_norm, data_gauss_aug_norm = normalization_fun(data_val, data_aug, data_gauss_val, data_gauss_aug, 0.01)


In [37]:

r.shuffle(data_val_norm)


In [ ]:
r.shuffle(data_gauss_val_norm)
r.shuffle(data_aug_norm)
r.shuffle(data_gauss_aug_norm)

## Define GPU device ##

In [5]:
gpu = tf.config.list_physical_devices('GPU')[0]
print(gpu)
tf.config.experimental.set_memory_growth(gpu, True)
gpu = tf.device('GPU:0/') 

base_dir = r'C:\Users\roumba\Documents\Software\deep-events'
data_path = base_dir+ r'\Norm_data'
model_path = base_dir+ '\Models'

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [6]:
with gpu:
  nb_filters = 8
  firstConvSize = 9
  batch_size = [8, 16, 32, 256]
  model, history= {}, {}
  
  b=batch_size[1]
  model_name = 'ref_f%i_c%i_b%i'%(nb_filters, firstConvSize, b)
  print('Model:', model_name)
  model[model_name] = training.create_model(nb_filters, firstConvSize)
  history[model_name] = training.train_model(model[model_name], data_aug_norm, data_gauss_aug_norm, b, data_ratio)

Model: ref_f8_c9_b16
* Start Encoder Section *
* Start Center Section *
* Start Decoder Section *


NameError: name 'data_aug_norm' is not defined

## Saving of Data ##

In [36]:
folder_name = list(model.keys())

## SAVE AS .h5 ##

# name1=f'\{cell_type}_{microscope}_{bf_fl}_data_val'
# name2=f'\{cell_type}_{microscope}_{bf_fl}_data_gauss_val'
# name3=f'\{cell_type}_{microscope}_{bf_fl}_data_aug'
# name4=f'\{cell_type}_{microscope}_{bf_fl}_data_gauss_aug'

# util.save_h5(data_val_norm, data_path, name1)
# util.save_h5(data_gauss_val_norm, data_path, name2)
# util.save_h5(data_aug_norm, data_path, name3)
# util.save_h5(data_gauss_aug_norm, data_path, name4)

## SAVE AS .tiff ##

name1=f'{cell_type}_{microscope}_{bf_fl}_data_val.tiff'
name2=f'{cell_type}_{microscope}_{bf_fl}_data_gauss_val.tiff'
name3=f'{cell_type}_{microscope}_{bf_fl}_data_aug.tiff'
name4=f'{cell_type}_{microscope}_{bf_fl}_data_gauss_aug.tiff'

imageio.mimwrite(name1, (data_val).astype(np.float64))
imageio.mimwrite(name2, (data_gauss_val_norm).astype(np.float64))
imageio.mimwrite(name3, (augmentation_data).astype(np.float64))
imageio.mimwrite(name4, (data_gauss_aug_norm).astype(np.float64))


In [12]:
print(all_image_array.shape)
writer = imageio.get_writer("all_images.tiff")
print(writer)
imageio.mimwrite("all_images.tiff", all_image_array[:4].astype(np.uint16),format=None, photometric='minisblack'})

(185, 256, 256)


TypeError: TiffFormat.Writer._open() got an unexpected keyword argument 'kwargs'

In [ ]:
util.save_model(model, model_path, [f'model']*len(model), folder_name)
util.save_pkl(history, model_path, [f'history']*len(model), folder_name)


Saving C:\Users\roumba\Documents\Software\deep-events\Modelsref_f8_c9_b16/model.h5
Done.

Saving C:\Users\roumba\Documents\Software\deep-events\Modelsref_f8_c9_b16/history
Done.
